<a href="https://colab.research.google.com/github/sushanedulloo20/Algorithms/blob/main/Part2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
classifier("I am really excited and depressed")

[{'label': 'NEGATIVE', 'score': 0.9968768358230591}]

In [5]:
from transformers import AutoTokenizer

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
tokenizer("Hi my god dammn you so hot",padding=True)

{'input_ids': [101, 7632, 2026, 2643, 5477, 2213, 2078, 2017, 2061, 2980, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
pt_batch = tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",
)
pt_batch

{'input_ids': tensor([[  101,  2057,  2024,  2200,  3407,  2000,  2265,  2017,  1996,   100,
         19081,  3075,  1012,   102],
        [  101,  2057,  3246,  2017,  2123,  1005,  1056,  5223,  2009,  1012,
           102,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])}

In [10]:
from transformers import AutoModelForSequenceClassification

model_name = "google-bert/bert-base-uncased"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SequenceClassifierOutput(loss=None, logits=tensor([[ 0.9076, -0.1745],
        [ 0.9545, -0.3294]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

tensor([[0.7469, 0.2531],
        [0.7831, 0.2169]], grad_fn=<SoftmaxBackward0>)


{'sequence': 'I have a problem with my iphone that needs to be resolved asap!!',
 'labels': ['no sex', 'sex'],
 'scores': [0.6880368590354919, 0.31196320056915283]}

In [2]:
!pip install datasets

In [3]:
import datasets
import torch
from nltk.translate import bleu_score
from tqdm import tqdm
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Load the WMT 2016 dataset
validation_dataset = datasets.load_dataset("wmt16", "de-en", split="validation")
test_dataset = datasets.load_dataset("wmt16", "de-en", split="test")

# Load the t5-small model and tokenizer
model_name = "google-t5/t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Define the prefix for zero-shot translation
prefix = "translate English to German: "

# Function to generate translations
def generate_translation(dataset):
    translations = []
    for example in tqdm(dataset, desc="Translating", unit=" examples"):

        german_text = example["translation"]["en"]
        input_text = prefix + german_text
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
        outputs = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
        translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        translations.append(translated_text)
    return translations

# Generate translations for validation and test sets
validation_translations = generate_translation(validation_dataset)
test_translations = generate_translation(test_dataset)

# Prepare reference and candidate sentences for BLEU score calculation
validation_references = [[example["translation"]["de"]] for example in validation_dataset]
test_references = [[example["translation"]["de"]] for example in test_dataset]

# Function to calculate BLEU score for a specific n-gram
def calculate_bleu_n(reference_corpus, translation_corpus, n):
    return bleu_score.corpus_bleu(reference_corpus, translation_corpus, weights=(1. / n,))

# Calculate BLEU-1, BLEU-2, BLEU-3, and BLEU-4 scores for validation and test sets
validation_bleu1_score = calculate_bleu_n(validation_references, validation_translations, 1)
validation_bleu2_score = calculate_bleu_n(validation_references, validation_translations, 2)
validation_bleu3_score = calculate_bleu_n(validation_references, validation_translations, 3)
validation_bleu4_score = calculate_bleu_n(validation_references, validation_translations, 4)

test_bleu1_score = calculate_bleu_n(test_references, test_translations, 1)
test_bleu2_score = calculate_bleu_n(test_references, test_translations, 2)
test_bleu3_score = calculate_bleu_n(test_references, test_translations, 3)
test_bleu4_score = calculate_bleu_n(test_references, test_translations, 4)

# Print BLEU scores
print("Validation BLEU-1 Score:", validation_bleu1_score)
print("Validation BLEU-2 Score:", validation_bleu2_score)
print("Validation BLEU-3 Score:", validation_bleu3_score)
print("Validation BLEU-4 Score:", validation_bleu4_score)

print("Test BLEU-1 Score:", test_bleu1_score)
print("Test BLEU-2 Score:", test_bleu2_score)
print("Test BLEU-3 Score:", test_bleu3_score)
print("Test BLEU-4 Score:", test_bleu4_score)


cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

...

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Translating: 100%|██████████| 2999/2999 [1:27:33<00:00,  1.75s/ examples]


Validation BLEU-1 Score: 0.8660011065290634
Validation BLEU-2 Score: 0.9285323950285187
Validation BLEU-3 Score: 0.9503639004236857
Validation BLEU-4 Score: 0.9614713924828087
Test BLEU-1 Score: 0.8716829824728705
Test BLEU-2 Score: 0.9287078393685444
Test BLEU-3 Score: 0.9485333960634894
Test BLEU-4 Score: 0.9586043237148643


# Cumulative scores

In [12]:
def calculate_bleu_n(reference_corpus, translation_corpus, n):
    if n==1:
      weights=(1,0,0,0)
    elif n==2:
      weights=(0.5,0.5,0,0)
    elif n==3:
      weights=(1/3,1/3,1/3,0)
    elif n==4:
      weights=(0.25,0.25,0.25,0.25)
    return bleu_score.corpus_bleu(reference_corpus, translation_corpus, weights=weights)

# Calculate BLEU-1, BLEU-2, BLEU-3, and BLEU-4 scores for validation and test sets
validation_bleu1_score = calculate_bleu_n(validation_references, validation_translations, 1)
validation_bleu2_score = calculate_bleu_n(validation_references, validation_translations, 2)
validation_bleu3_score = calculate_bleu_n(validation_references, validation_translations, 3)
validation_bleu4_score = calculate_bleu_n(validation_references, validation_translations, 4)

test_bleu1_score = calculate_bleu_n(test_references, test_translations, 1)
test_bleu2_score = calculate_bleu_n(test_references, test_translations, 2)
test_bleu3_score = calculate_bleu_n(test_references, test_translations, 3)
test_bleu4_score = calculate_bleu_n(test_references, test_translations, 4)

# Print BLEU scores
print("Validation BLEU-1 Score:", validation_bleu1_score)
print("Validation BLEU-2 Score:", validation_bleu2_score)
print("Validation BLEU-3 Score:", validation_bleu3_score)
print("Validation BLEU-4 Score:", validation_bleu4_score)

print("Test BLEU-1 Score:", test_bleu1_score)
print("Test BLEU-2 Score:", test_bleu2_score)
print("Test BLEU-3 Score:", test_bleu3_score)
print("Test BLEU-4 Score:", test_bleu4_score)

Validation BLEU-1 Score: 0.8660011065290634
Validation BLEU-2 Score: 0.7870934277787213
Validation BLEU-3 Score: 0.7218977276645724
Validation BLEU-4 Score: 0.6707315953509069
Test BLEU-1 Score: 0.8716829824728705
Test BLEU-2 Score: 0.8015556586843793
Test BLEU-3 Score: 0.7434569741468052
Test BLEU-4 Score: 0.6972965249654124


# Calculating the Indivdual BLEU Scores

In [6]:
def calculate_bleu_n(reference_corpus, translation_corpus, n):
    if n==1:
      weights=(1,0,0,0)
    elif n==2:
      weights=(0,1,0,0)
    elif n==3:
      weights=(0,0,1,0)
    else:
      weights=(0,0,0,1)
    return bleu_score.corpus_bleu(reference_corpus, translation_corpus, weights=weights)

# Calculate BLEU-1, BLEU-2, BLEU-3, and BLEU-4 scores for validation and test sets
validation_bleu1_score = calculate_bleu_n(validation_references, validation_translations, 1)
validation_bleu2_score = calculate_bleu_n(validation_references, validation_translations, 2)
validation_bleu3_score = calculate_bleu_n(validation_references, validation_translations, 3)
validation_bleu4_score = calculate_bleu_n(validation_references, validation_translations, 4)

test_bleu1_score = calculate_bleu_n(test_references, test_translations, 1)
test_bleu2_score = calculate_bleu_n(test_references, test_translations, 2)
test_bleu3_score = calculate_bleu_n(test_references, test_translations, 3)
test_bleu4_score = calculate_bleu_n(test_references, test_translations, 4)

# Print BLEU scores
print("Validation BLEU-1 Score:", validation_bleu1_score)
print("Validation BLEU-2 Score:", validation_bleu2_score)
print("Validation BLEU-3 Score:", validation_bleu3_score)
print("Validation BLEU-4 Score:", validation_bleu4_score)

print("Test BLEU-1 Score:", test_bleu1_score)
print("Test BLEU-2 Score:", test_bleu2_score)
print("Test BLEU-3 Score:", test_bleu3_score)
print("Test BLEU-4 Score:", test_bleu4_score)

Validation BLEU-1 Score: 0.8660011065290634
Validation BLEU-2 Score: 0.7153756033124258
Validation BLEU-3 Score: 0.607259688146411
Validation BLEU-4 Score: 0.5379823579987583
Test BLEU-1 Score: 0.8716829824728705
Test BLEU-2 Score: 0.7370701125152979
Test BLEU-3 Score: 0.6395877697660285
Test BLEU-4 Score: 0.5753107433087195


## Loading The translations and Refrences

In [10]:
import pickle
pickle.dump(validation_references,open("validation_refrences.pkl","wb"))
pickle.dump(validation_translations,open("validation_translations.pkl","wb"))
pickle.dump(test_references,open("test_refrences.pkl","wb"))
pickle.dump(test_translations,open("test_translations.pkl","wb"))

# Meteor Calculation

In [15]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00


In [16]:
import evaluate
meteor=evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [19]:
meteor_validation=meteor.compute(predictions=validation_translations,references=validation_references)
print(f"METEOR Validation : {meteor_validation['meteor']}")
meteor_test=meteor.compute(predictions=test_translations,references=test_references)
print(f"METEOR Test : {meteor_test['meteor']}")

METEOR Validation : 0.5518401805312027
METEOR Test : 0.5904659873087972


# Computing BERT Score

In [21]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 961.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [22]:
import evaluate
bert_score=evaluate.load("bertscore")

In [25]:
bert_validation=bert_score.compute(predictions=validation_translations,references=validation_references,lang="de")
print(f"BERTSCORE Validation : {bert_validation}")
bert_test=bert_score.compute(predictions=test_translations,references=test_references,lang="de")
print(f"BERTSCORE Test : {bert_test}")

BERTSCORE Validation : {'precision': [0.9225234389305115, 0.8907819390296936, 0.9487404227256775, 0.8868697881698608, 0.9370797276496887, 0.8531867265701294, 0.9534709453582764, 0.8689139485359192, 0.8976612091064453, 0.9476878046989441, 0.8262912631034851, 0.9396442174911499, 0.9311310648918152, 0.8699209690093994, 0.8598195910453796, 0.8546431064605713, 0.9577564597129822, 0.7675563097000122, 0.8691653609275818, 0.9007344841957092, 0.8464642763137817, 0.9132484793663025, 0.8719236254692078, 0.7746807932853699, 0.8293420076370239, 0.8306722640991211, 0.9202014803886414, 0.8604443669319153, 0.8671427965164185, 0.8308663964271545, 0.9312722682952881, 0.8518227934837341, 0.9229822158813477, 0.875878632068634, 0.8353450298309326, 0.9262533783912659, 0.9005718231201172, 0.9004573822021484, 0.8026542067527771, 0.9676380157470703, 0.9380099773406982, 0.9416096806526184, 0.8678898811340332, 0.9450990557670593, 0.8694037199020386, 0.9142063856124878, 0.9318264722824097, 0.8650988340377808, 0.8

In [26]:
print(f'''Bert Score Validation:
          F1 SCORE: {bert_validation["f1"]}
          Precision:{bert_validation["precision"]}
          Recall:   {bert_validation["recall"]} ''')
print(f'''Bert Score Test:
          F1 SCORE: {bert_test["f1"]}
          Precision:{bert_test["precision"]}
          Recall:   {bert_test["recall"]} ''')

Bert Score Validation:
          F1 SCORE: [0.9118711352348328, 0.8656713366508484, 0.9472792744636536, 0.8706494569778442, 0.9193931818008423, 0.8585269451141357, 0.9541038274765015, 0.8658657073974609, 0.8837403655052185, 0.9504255056381226, 0.811790406703949, 0.936506450176239, 0.9215840697288513, 0.877633273601532, 0.8658899068832397, 0.8549880981445312, 0.9297122955322266, 0.774607241153717, 0.8463473320007324, 0.898991048336029, 0.8377185463905334, 0.9038700461387634, 0.8810357451438904, 0.775692343711853, 0.8341607451438904, 0.8346177339553833, 0.9164811372756958, 0.8607688546180725, 0.8704962134361267, 0.8432490825653076, 0.9190680384635925, 0.8614271283149719, 0.9261685609817505, 0.862098753452301, 0.8265120983123779, 0.9194155931472778, 0.907621443271637, 0.8966445326805115, 0.7947973608970642, 0.9746332168579102, 0.9117384552955627, 0.9208521842956543, 0.8571739196777344, 0.9518145322799683, 0.8471116423606873, 0.913990318775177, 0.9224715232849121, 0.8339073657989502, 0.875

# Pipeline for PART 2B

In [14]:
import datasets
import torch
from nltk.translate import bleu_score
from tqdm import tqdm
from transformers import T5ForConditionalGeneration, T5Tokenizer
src_input=input("Enter a language in english: ")
prefix="translate English to German: "
inp_text=prefix+src_input
model_name = "google-t5/t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
tokenizer = T5Tokenizer.from_pretrained(model_name)
input_ids = tokenizer.encode(inp_text, return_tensors="pt", max_length=512, truncation=True).to(device)
outputs = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
translated_text



Enter a language in english: I am very handsome


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'Ich bin sehr schön'